# **1.감성분석(Sentiment_analysis)**

### 전체 상품 리뷰 to 긍정, 부정 Table

In [ ]:
#전체리뷰 To csv
from tqdm import tqdm
prod_list= pd.read_csv('total_up.csv',encoding='utf-8-sig')

df_all_review=pd.DataFrame()

for id in tqdm(prod_list['product_id']):
    df_rev= pd.read_csv('goods_review_table_v2/reviews_'+id+'.csv',encoding='utf-8-sig')
    df_rev.dropna(subset=['text'],inplace=True) 

    if len(df_all_review.columns)==0: 
        df_all_review=df_rev
    else: 
        df_all_review=pd.concat([df_all_review,df_rev],ignore_index=True)

print(df_all_review.shape)

100%|██████████| 83/83 [00:01<00:00, 61.13it/s]

(50162, 10)


# 전체상품 전체 댓글 감성분석

In [ ]:
# 전처리 함수 OKT
from konlpy.tag import Okt
import re
def preprocessing(text):
    with open('stop_words.txt','rt','utf-8-sig') as f:
        stop_words = f.read().split(',')
    okt = Okt()
    rev_text=re.sub('[^가-힣\\s]','',text)
    word_review = okt.pos(rev_text,stem=True)
    word_review = [(token, pos) for token, pos in word_review if not token in stop_words and len(token)>1]
    word_review = [token for token, pos in word_review if pos in ['Noun', 'Verb', 'Adjective']]
    return word_review

In [ ]:
# 전처리 함수 ckonlpy
from konlpy.tag import Okt
import ckonlpy
from ckonlpy.tag import Twitter as OriginalTwitter
import re
def preprocessing(text):
    with open('stop_words.txt','r',encoding='utf-8-sig') as f:
        stop_words = f.read().split(',')
    okt = OriginalTwitter()
    rev_text=re.sub('[^가-힣\\s]','',text)
    word_review = okt.pos(rev_text)
    word_review = [(token, pos) for token, pos in word_review if not token in stop_words and len(token)>1]
    word_review = [token for token, pos in word_review if pos in ['Noun', 'Verb', 'Adjective']]
    return word_review

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd
#전체리뷰 To csv
from tqdm import tqdm
prod_list= pd.read_csv('total_up.csv',encoding='utf-8-sig')

df_all_review=pd.DataFrame()

for id in tqdm(prod_list['product_id']):
    df_rev= pd.read_csv('goods_review_table_v2/reviews_'+id+'.csv',encoding='utf-8-sig')
    df_rev.dropna(subset=['text'],inplace=True) 
    if len(df_all_review.columns)==0: 
        df_all_review=df_rev
    else: 
        df_all_review=pd.concat([df_all_review,df_rev],ignore_index=True)

# all_tfidf = TfidfVectorizer(tokenizer=preprocessing, max_features=2000, min_df=5, max_df=0.5,ngram_range=(1,2))
all_tfidf = TfidfVectorizer(tokenizer=preprocessing, max_features=2000, min_df=5, max_df=0.5)

all_train_tfidf = all_tfidf.fit_transform(df_all_review['text'])

100%|██████████| 83/83 [00:01<00:00, 80.67it/s]
c:\Users\vzb88\AppData\Local\Programs\Python\Python310\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
params = {
    'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
}
logi_clf = LogisticRegression()
logi_clf.fit(all_train_tfidf,int(df_all_review['label']))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
from operator import attrgetter, index, itemgetter


pos_words=logi_clf.coef_[0]
temp=[]
for i,val in enumerate(pos_words): temp.append((val,i))
coef_pos_words=sorted(temp,reverse=True)
coef_pos_words

[(2.530138998779446, 1477),
 (2.518521782498474, 1675),
 (2.135309609804235, 1430),
 (2.067930818355735, 50),
 (2.0148439093189605, 1846),
 (1.9064813774969924, 44),
 (1.7754400550677472, 1567),
 (1.7551513646861905, 1121),
 (1.6777953901672207, 1694),
 (1.6097886355452116, 1351),
 (1.6066223746495176, 1293),
 (1.605680606693626, 1682),
 (1.5661238522366698, 1790),
 (1.561206575712985, 1762),
 (1.5034625988557389, 449),
 (1.4877544811588117, 1736),
 (1.4864425490796593, 1795),
 (1.4687314257342263, 835),
 (1.4174400197381305, 1416),
 (1.3646947631441428, 974),
 (1.3575683273156178, 1110),
 (1.3264107737120976, 1398),
 (1.3213122796552939, 1570),
 (1.3113324082552664, 1124),
 (1.3055508489879943, 1285),
 (1.3045213309518997, 1432),
 (1.2898646268450098, 1431),
 (1.2805148099099157, 359),
 (1.2802857806467363, 523),
 (1.2663998774468763, 335),
 (1.2616825085517762, 831),
 (1.261360108569941, 450),
 (1.256357227592453, 1925),
 (1.2454106918505277, 1769),
 (1.2338329587052732, 75),
 (1.230

In [ ]:
text_data_dict={}
for key in all_tfidf.vocabulary_:
    value = all_tfidf.vocabulary_[key]
    text_data_dict[value]=key
text_data_dict

{1189: '자극',
 624: '부드럽다',
 1208: '자극 부드럽다',
 629: '부드럽다 좋다',
 1309: '젛아',
 1854: '평생',
 512: '반신반의',
 1052: '여드름',
 1798: '트러블',
 259: '꾸준하다',
 413: '맑다',
 891: '아니다',
 453: '문제',
 1663: '촉촉하다',
 1968: '흡수',
 535: '보습',
 1: '가볍다',
 669: '사계절',
 939: '않다',
 1851: '편하다',
 1306: '정착',
 1715: '추천',
 1820: '트러블 아니다',
 1707: '촉촉하다 흡수',
 1977: '흡수 보습',
 536: '보습 가볍다',
 671: '사계절 좋다',
 1450: '좋다 자극',
 1223: '자극 않다',
 984: '않다 아니다',
 1452: '좋다 자극 않다',
 397: '만족스럽다',
 434: '무난',
 1265: '장점',
 514: '반하다',
 1921: '효과',
 236: '기대하다',
 1571: '진정',
 1840: '특별하다',
 349: '듬뿍',
 505: '믿다',
 436: '무난 무난',
 1243: '자극 효과',
 1926: '효과 기대하다',
 438: '무난 보습',
 578: '보습 진정',
 1464: '좋다 좋다',
 1386: '좋다 보습',
 1466: '좋다 좋다 보습',
 35: '각질',
 1899: '확실하다',
 676: '산뜻하다',
 316: '대비',
 656: '비싸다',
 1608: '진정 진정',
 268: '꾸준하다 효과',
 1962: '효과 확실하다',
 319: '대비 비싸다',
 659: '비싸다 좋다',
 93: '건성',
 1528: '좋아지다',
 1088: '여름',
 115: '건성 자극',
 1006: '않다 좋아지다',
 1105: '여름 좋다',
 597: '복합',
 1121: '예민하다',
 1119: '예민',
 793: '순하다',
 

In [ ]:
dic_coef={
    'word':[],
    'tfidf':[]
}
for val,ind in coef_pos_words:
    dic_coef['word'].append(text_data_dict[ind])
    dic_coef['tfidf'].append(val)
df_coef=pd.DataFrame(dic_coef)
df_coef.to_csv('allReview_coef.csv',encoding='utf-8-sig')
df_coef


,word,tfidf
0,좋다 좋다 흡수,2.530139
1,촉촉하다 만족하다,2.518522
2,좋다 않다,2.135310
3,각질 진정,2.067931
4,퍽퍽,2.014844
...,...,...
1995,비슷하다,-2.114249
1996,괜찮다 건조하다,-2.208527
1997,건조하다 좋다,-2.318519
1998,성분 보습,-2.482916


In [ ]:
with open('allReview_pos_dict.txt','rt') as f: pos_dic=f.read().split(',')
with open('allReview_neg_dict.txt','rt') as f: neg_dic=f.read().split(',')
temp =[]
for i in df_coef['word']:
    if i not in pos_dic and not i in neg_dic:
        temp.append(i)


with open('allReview_coef_words2.txt','wt') as f:
    f.write(','.join(temp))

In [ ]:
with open('부정어.txt','rt') as f: new_dic=f.read().split(',')
with open('allReview_neg_dict.txt','rt') as f: neg_dic=f.read().split(',')
print(len(neg_dic))
for wd in new_dic:
    if not wd in neg_dic:neg_dic.append(wd)
with open('allReview_neg_dict.txt','wt') as f: f.write(','.join(neg_dic))
print(len(neg_dic))

159
565


# 상품별 댓글 긍정,부정 사전 기반 라벨링

In [ ]:
# 라벨링 -> csv
from cmath import nan
from tqdm import tqdm
import pandas as pd

prod_lst=pd.read_csv('total_up.csv',encoding='utf-8-sig')

with open('allReview_pos_dict.txt','rt') as f: pos_dic=f.read().split(',')
with open('allReview_neg_dict.txt','rt') as f: neg_dic=f.read().split(',')


for id in tqdm(prod_lst['product_id']):
    df_rev= pd.read_csv('goods_review_table_v2/reviews_'+id+'.csv',encoding='utf-8-sig')
    df_rev.dropna(subset=['text'],inplace=True)
    label=[]
    for i,row in df_rev.iterrows():
        if not type(row['text'])==str:
            label.append(nan)
            print(label)
            continue
        txt = preprocessing(row['text'])
        p_cnt=0
        n_cnt=0 
        for wrd in txt:
            if wrd in pos_dic: p_cnt=p_cnt+1
            if wrd in neg_dic: n_cnt=n_cnt+1
        if p_cnt-n_cnt+int(row['rev_score'])/4>=0 : label.append(1)
        if p_cnt-n_cnt+int(row['rev_score'])/4<0 :
            flag=True
            for wrd in txt[:-9:-1]:
                if wrd in pos_dic:
                    label.append(1)
                    flag=False
                    break
            if flag :label.append(0)
    df_rev['label']=label
    df_rev.to_csv('goods_review_table_v2/reviews_'+id+'.csv',index=False,encoding='utf-8-sig')


100%|██████████| 83/83 [10:04<00:00,  7.28s/it]


In [ ]:
# 긍정, 부정 리뷰 카운트
from cmath import isnan, nan
import numpy as np


prod_lst=pd.read_csv('total_up.csv',encoding='utf-8-sig')
pos_cnt=0
neg_cnt=0

star_p=0
star_n=0
for id in tqdm(prod_lst['product_id']):
    df_rev= pd.read_csv('goods_review_table_v2/reviews_'+id+'.csv',encoding='utf-8-sig')
    df_rev.dropna(subset=['text'],inplace=True)
    
    for score in df_rev['label']:
        if isnan(score):continue
        if int(score)==1: pos_cnt=pos_cnt+1
        if int(score)==0: neg_cnt=neg_cnt+1

    for score in df_rev['rev_score']:
        if score//10: star_p=star_p+1
        else: star_n=star_n+1

print('Positive revies: ',pos_cnt,', Negative reviews: ',neg_cnt)
print('5 stars: ',star_p,', 1to4 stars: ',star_n)

100%|██████████| 83/83 [00:01<00:00, 79.29it/s]

Positive revies:  47851 , Negative reviews:  1985
5 stars:  40227 , 1to4 stars:  9609


In [ ]:
# 댓글파일 'label' column drop
from tqdm import tqdm
import pandas as pd 
prod_lst=pd.read_csv('total_up.csv',encoding='utf-8-sig')
for id in tqdm(prod_lst['product_id']):
    df_rev= pd.read_csv('goods_review_table_v2/reviews_'+id+'.csv',encoding='utf-8-sig')
    df_rev=df_rev.drop(columns=['label'])
    df_rev.to_csv('goods_review_table_v2/reviews_'+id+'.csv',index=False,encoding='utf-8-sig')

100%|██████████| 83/83 [00:02<00:00, 36.02it/s]


In [ ]:
# 라벨링 한 긍정, 부정 댓글 비율
from tqdm import tqdm
import pandas as pd 
import numpy as np
prod_lst=pd.read_csv('total_up.csv',encoding='utf-8-sig')
p_cnt=0
n_cnt=0
dic_txt={
    
}
for id in tqdm(prod_lst['product_id']):
    df_rev= pd.read_csv('goods_review_table_v2/reviews_'+id+'.csv',encoding='utf-8-sig')
    for row in df_rev['label']:
        if isnan(row):continue
        if row==1:p_cnt=p_cnt+1
        if row==0:n_cnt=n_cnt+1
print(p_cnt,n_cnt)


100%|██████████| 83/83 [00:00<00:00, 100.02it/s]

47851 1985


In [ ]:
# 성분 등급점수 scale
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
df_ing = pd.read_csv('ing_rating.csv',encoding='utf-8-sig')
score = [[5-i] for i in df_ing['score']]
# print(score.shape())
std = StandardScaler()
std.fit(score)
scaled_score = std.transform(score)
score = [(i[0]) for i in scaled_score]
df_ing['score_standardScaled']=score
df_ing.to_csv('ing_rating_std.csv',encoding='utf-8-sig')

In [ ]:
# 상품별 리뷰 긍정 비율 total_up add
df_product = pd.read_csv('total_up.csv',encoding='utf-8-sig')
rev_score=[]
for id in df_product['product_id']:
    df_rev= pd.read_csv('goods_review_table_v2/reviews_'+id+'.csv',encoding='utf-8-sig')
    label_cnt=0
    label_sum=0
    for label in df_rev['label']:
        if isnan(label):continue
        label_sum=label_sum+int(label)
        label_cnt=label_cnt+1
    rev_score.append(label_sum/label_cnt)
df_product['positive_rate']=rev_score
df_product.to_csv('total_up_v1.csv',encoding='utf-8-sig')

In [ ]:
# 상품별 리뷰 부정cnt total_up add
df_product = pd.read_csv('total_up_v1.csv',encoding='utf-8-sig')
rev_score=[]
for id in df_product['product_id']:
    df_rev= pd.read_csv('goods_review_table_v2/reviews_'+id+'.csv',encoding='utf-8-sig')
    neg_cnt=0
    for label in df_rev['label']:
        if isnan(label):continue
        if label==0:neg_cnt=neg_cnt+1
    rev_score.append(neg_cnt)
df_product['negative_cnt']=rev_score
df_product.to_csv('total_up_v2_negCnt.csv',encoding='utf-8-sig')

# **2.LDA 토픽모델링**

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components =3, 
                                max_iter = 20,
                                learning_method='online',
                                random_state=100,
                                verbose=True)
lda_output = lda.fit_transform(tfidf)

iteration: 1 of max_iter: 20
iteration: 2 of max_iter: 20
iteration: 3 of max_iter: 20
iteration: 4 of max_iter: 20
iteration: 5 of max_iter: 20
iteration: 6 of max_iter: 20
iteration: 7 of max_iter: 20
iteration: 8 of max_iter: 20
iteration: 9 of max_iter: 20
iteration: 10 of max_iter: 20
iteration: 11 of max_iter: 20
iteration: 12 of max_iter: 20
iteration: 13 of max_iter: 20
iteration: 14 of max_iter: 20
iteration: 15 of max_iter: 20
iteration: 16 of max_iter: 20
iteration: 17 of max_iter: 20
iteration: 18 of max_iter: 20
iteration: 19 of max_iter: 20
iteration: 20 of max_iter: 20


In [ ]:
df_lda

,보습,진정,저자극
0,0.194700,0.098545,0.706755
1,0.458224,0.411911,0.129865
2,0.232751,0.300680,0.466570
3,0.755645,0.069421,0.174934
4,0.666663,0.166669,0.166668
...,...,...,...
47846,0.508943,0.107685,0.383372
47847,0.356671,0.441934,0.201395
47848,0.419902,0.113423,0.466675
47849,0.541009,0.366076,0.092915


In [ ]:
pos_rev

,relative_id,goods_name,date,text,age,gender,rev_score,skinTrouble,product_id,label,preprocessed
0,A000000167276,피캄 베리어 사이클 KMX 토너 200ml,2022.07.23,팩으로도 닥토로도 사용해봤는데 사용하고 오히려 피부가 뒤집어져서 당황했네요ㅠ\n화농...,20,F,6,NaN,A000000167276,1,뒤집어지다 당황 화농성 좋다 아지 좋다 아지
1,A000000167276,피캄 베리어 사이클 KMX 토너 200ml,2022.07.20,수부지 복합성 피부이면셔 민감성 피부인데도 불구하고 너무 순해서 자극이 하나도 없어...,10,F,10,NaN,A000000167276,1,수부지 복합성 민감성 불구 자극 좋다 여름 무겁다 않다 가볍다 해소 수분 좋다
2,A000000167276,피캄 베리어 사이클 KMX 토너 200ml,2022.07.20,크기가 작아서 금방써서 속상하지만 진정도 되고 수분감도 좋아서 재구매 하려고요~ \...,30,F,10,NaN,A000000167276,1,작다 진정 수분감 좋다 재구매 제형 맑은 촉촉하다 닦토
3,A000000167276,피캄 베리어 사이클 KMX 토너 200ml,2022.07.18,지성이여서 여름에 유수분 밸런스 맞추기가 힘들었는데 이 토너는 촉촉하면서 산뜻해서 ...,40,F,10,NaN,A000000167276,1,지성 여름 추기 힘들다 촉촉하다 산뜻 흡수 좋다 도움 여드름 진정 재구매 얇다 분간...
4,A000000167276,피캄 베리어 사이클 KMX 토너 200ml,2022.07.17,잘 쓰고 있습니다. 아직 사용한지 초반이라 잘 모르겠지만 쓰다보면 효과가 생기겠죠?,20,F,10,NaN,A000000167276,1,효과
...,...,...,...,...,...,...,...,...,...,...,...
47846,A000000002839,바이오더마 센시비오 토너 250ml,2020.04.24,원래쓰던거라서 계속쓰는데 진짜 순하고 좋아요. 친구추천으로 첨썼는데 피뷰자극도 덜하...,30,F,10,NaN,A000000002839,1,좋다 추천 자극 어진
47847,A000000002839,바이오더마 센시비오 토너 250ml,2020.04.23,피부가 예민한 편이라 아무제품이나 쓰기가 무서웠는데 이 제품은 자극도 없고 순해서 ...,20,F,8,NaN,A000000002839,1,예민 무섭다 자극 좋다 건성 좋다 수분크림 좋다
47848,A000000002839,바이오더마 센시비오 토너 250ml,2020.04.23,피부를 진정시키는데 짱인것같아요.\n피부가 민감할때 스킨팩으로 쓰면 금방 진정됩니다,20,F,10,NaN,A000000002839,1,진정 민감 스킨팩 진정
47849,A000000002839,바이오더마 센시비오 토너 250ml,2020.04.23,세일기간에 좋은제품 싸게 잘 샀습니다. 클렌징 워터도 잘 쓰고있는데. 토너도 처음 ...,30,F,10,"C01,C02,C03,C09",A000000002839,1,좋다 싸다 좋다 가볍다 족하 감사


In [ ]:
df_lda = pd.DataFrame(lda_output)
df_lda.columns = ['보습', '진정', '저자극']

In [ ]:
pos_rev = pos_rev.join(df_lda)

In [ ]:
corpus_df['보습'] = None
corpus_df['진정'] = None
corpus_df['저자극'] = None

In [ ]:
for i in pos_rev['product_id'].unique():
    corpus_df.loc[corpus_df['id']==i, '보습'] = pos_rev[pos_rev['product_id']==i][['보습', '진정', '저자극']].mean()[0]
    corpus_df.loc[corpus_df['id']==i, '진정'] = pos_rev[pos_rev['product_id']==i][['보습', '진정', '저자극']].mean()[1]
    corpus_df.loc[corpus_df['id']==i, '저자극']= pos_rev[pos_rev['product_id']==i][['보습', '진정', '저자극']].mean()[2]
corpus_df

,Unnamed: 0,Unnamed: 0.1,0,id,보습,진정,저자극
0,0,0,순 부드럽다 트러블 일으키다 없다 트러블 범벅 좋다 화장솜 듬뿍 팩 팩 떼다 이상 ...,A000000002839,0.352698,0.283684,0.363618
1,1,1,스킨팩 좋다 묽다 제형 니지 속건조 좋다 산뜻 가볍다 계절 좋다 자극 없다 수분 부...,A000000004434,0.307483,0.311843,0.380673
2,2,2,작년 뭣 다가 개이득 정가 아쉽다 일어 유 폭발 지성인 추천 닦토 해주 깔다 끄다 ...,A000000005766,0.420292,0.273563,0.306145
3,3,3,흐르다 물 제형 순 쿨링 감 베이스 전 좋다 좋다 트러블 심하다 편 별로 관심 없다...,A000000007413,0.334513,0.301927,0.363561
4,4,4,라로슈포제 애정 그렇다 주변인 들다 추천 서다 남자친구 추천 남자 들다 지성 거나 ...,A000000010474,0.386164,0.266157,0.347679
...,...,...,...,...,...,...,...
78,78,78,평생 대용량 좋다 진정 데 탑 물 흡수 빨르다 로만 대용량 나와라 노래 부르다 대용...,A000000162273,0.416308,0.240059,0.343633
79,79,79,막상 사고 맞다 고민 막상 서다 물 지도 않다 손 덜다 주르륵 떨다 없다 시원 놀라...,A000000162662,0.313634,0.29107,0.395296
80,80,80,남동생 찾다 다가 좋다 얘기 들다 전이 맞다 좋다 늘다 히알루론산 맞다 거 좁쌀 여...,A000000163493,0.301859,0.409588,0.288553
81,81,81,좁다 살 여드름 여드름 찾다 걸 추 천해 주더 거 여드름 진정 스킨팩 해주 발라 좁...,A000000165069,0.434722,0.248892,0.316386


In [ ]:
corpus_df.to_csv('/content/drive/MyDrive/corpus_df.csv', encoding='utf-8-sig')

In [ ]:
corpus_df = pd.read_csv('/content/drive/MyDrive/corpus_df.csv')

In [ ]:
corpus_df.loc[:2]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,0,id,보습,진정,저자극
0,0,0,0,순 부드럽다 트러블 일으키다 없다 트러블 범벅 좋다 화장솜 듬뿍 팩 팩 떼다 이상 ...,A000000002839,0.352698,0.283684,0.363618
1,1,1,1,스킨팩 좋다 묽다 제형 니지 속건조 좋다 산뜻 가볍다 계절 좋다 자극 없다 수분 부...,A000000004434,0.307483,0.311843,0.380673
2,2,2,2,작년 뭣 다가 개이득 정가 아쉽다 일어 유 폭발 지성인 추천 닦토 해주 깔다 끄다 ...,A000000005766,0.420292,0.273563,0.306145


In [ ]:
# LDA 시각화를 진행해봅시다. 
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vect, mds='tsne')  # tsne : 차원축소 기법
pyLDAvis.display(vis)
pyLDAvis.save_html(vis, f'/content/drive/MyDrive/text_pj/lda/final_lda.html')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 17.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=cee424988d882aecb8846028953686ff4ad9405a79b45a84ac60956bf68b26ca
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on